In [1]:
from src import efficientNet
from src import pre_process

from tensorflow.keras.preprocessing import image
import tensorflow as tf
from PIL import Image
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
Image.MAX_IMAGE_PIXELS = 3000000000 

train1 = pd.read_csv("../Final-IRONHACK-Project/data/train1.csv")
train_x, valid_x, train_y, valid_y = pre_process.train_valid_split(train1)

# 0. Data and instantiating
train_generator, validation_generator = pre_process.image_generator_for_B4()

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


`EfficientNet model:`

In [2]:
efficient_model = efficientNet.EfficientNet()
efficient_model.efficient_model()
efficient_model.unfreeze_model()

In [3]:
efficient_model = efficientNet.EfficientNet()

architecture = efficient_model.efficient_model()

# architecture.summary()

efficient_model.unfreeze_model()

In [4]:
efficient_model.model.summary()

Model: "EfficientNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 500, 500, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_2 (Rescaling)        (None, 500, 500, 3)  0           ['input_3[0][0]']                
                                                                                                  
 normalization_2 (Normalization  (None, 500, 500, 3)  7          ['rescaling_2[0][0]']            
 )                                                                                                
                                                                                       

In [5]:
# 1. Fitting the model
hist = efficient_model.train(train_generator, validation_generator, epochs = 3)
efficient_model.get_weights('../Final-IRONHACK-Project/data/Efficient_weights.h5')
class_names = list(validation_generator.class_indices.keys())
predictions = efficient_model.predict_efficientNet(validation_generator,class_names)

Epoch 1/3
32/32 [==============================] - 3335s 104s/step - loss: 0.9053 - accuracy: 0.4563 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/3
32/32 [==============================] - 3238s 101s/step - loss: 0.8507 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 3/3
1/1 [==============================] - 2s 2s/step


In [6]:
# 2. Evaluating the model
training_loss = hist.history['loss']
print("The training loss is of:", training_loss)
validation_loss = hist.history['val_loss']
print("The validation loss is of:", validation_loss)
training_accuracy = hist.history['accuracy']
print("The training accuracy is of:", training_accuracy)
validation_accuracy = hist.history['val_accuracy']
print("The validation accuracy is of:", validation_accuracy)


The training loss is of: [0.905329704284668, 0.8506633043289185, 0.8946544528007507]
The validation loss is of: [0.6931840181350708, 0.6932702660560608, 0.6931506991386414]
The training accuracy is of: [0.45625001192092896, 0.5, 0.4000000059604645]
The validation accuracy is of: [0.5, 0.5, 0.5]


`Predict on new images:`

In [7]:
# 3. Predicting on new data
efficient_model.load_weights('../Final-IRONHACK-Project/data/Efficient_weights.h5')


In [8]:
test_path = ["D:/bootcamp/test/006388_0.tif","D:/bootcamp/test/008e5c_0.tif", "D:/bootcamp/test/00c058_0.tif", "D:/bootcamp/test/01adc5_0.tif"]

preprocessed_images = []
for img_path in test_path:
    img = image.load_img(img_path, target_size=(380, 380)) 
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) 
    img_array /= 255.0  
    preprocessed_images.append(img_array)

for img in preprocessed_images:
    predictions = efficient_model.predict_on_data(img, class_names)
    print(predictions)

1/1 [==============================] - 1s 1s/step
[[0.49923462 0.50076544]]
1/1 [==============================] - 0s 160ms/step
[[0.49923462 0.50076544]]
1/1 [==============================] - 0s 164ms/step
[[0.49923462 0.50076544]]
1/1 [==============================] - 0s 163ms/step
[[0.49923462 0.50076544]]
